# 단속결과 / 구 혹은 동 면적
- 단속결과에 대한 구 혹은 동 면적을 구합니다.
- 동의 경우, 법정동인 경우만 다룹니다.

## 자동차 단속 및 처리 한땀한땀
- 2018 ~ 2021 의 자동차 단속 및 처리 내용
- https://www.data.go.kr/data/15063394/fileData.do

In [175]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [176]:
df_bad_cars = pd.read_excel(f'./externals/공공_데이터_포털/자동차_단속_및_처리_20231204120119.xlsx')
df_bad_cars.head(3)

/home/sdk/miniconda3/envs/MathAI/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,경찰서별(1),2018,2018.1,2018.2,2018.3,2018.4,2018.5,2018.6,2018.7,2018.8,...,2021.11,2021.12,2021.13,2021.14,2021.15,2021.16,2021.17,2021.18,2021.19,2021.20
0,경찰서별(1),총계,위반사항,위반사항,위반사항,위반사항,위반사항,위반사항,위반사항,위반사항,...,차종별,차종별,차종별,용도별,용도별,용도별,처리사항,처리사항,처리사항,처리사항
1,경찰서별(1),소계,신호위반,과속,안전운전 의무불이행,안전띠 미착용,중앙선 침범,음주운전,무면허,기타,...,화물차,이륜차,기타(특수차),사업용,비사업용,기타,형사 입건,즉심,통고처분,무인단속
2,합계,591625,93999,446367,5078,14223,5652,7682,2099,16525,...,88035,21822,5430,91184,672456,403,7781,6,62136,694120


In [177]:
df_bad_cars = df_bad_cars.T.reset_index()
df_bad_cars.head(3)

,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,경찰서별(1),경찰서별(1),경찰서별(1),합계,중부경찰서,동부경찰서,서부경찰서,남부경찰서,북부경찰서,수성경찰서,달서경찰서,성서경찰서,달성경찰서,강북경찰서
1,2018,총계,소계,591625,28116,83062,65465,33239,31891,98977,76418,38227,72389,63841
2,2018.1,위반사항,신호위반,93999,1618,13629,4703,3747,7589,20824,19523,7945,11716,2705


In [178]:
df_bad_cars.iloc[0, 0] = '년도'
df_bad_cars.iloc[0, 1] = '구분'
df_bad_cars.iloc[0, 2] = '경찰서별'
df_bad_cars.head(1)

,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,년도,구분,경찰서별,합계,중부경찰서,동부경찰서,서부경찰서,남부경찰서,북부경찰서,수성경찰서,달서경찰서,성서경찰서,달성경찰서,강북경찰서


In [179]:
df_bad_cars = df_bad_cars.rename(columns=df_bad_cars.iloc[0, :]).drop(0)
df_bad_cars.head(1)

,년도,구분,경찰서별,합계,중부경찰서,동부경찰서,서부경찰서,남부경찰서,북부경찰서,수성경찰서,달서경찰서,성서경찰서,달성경찰서,강북경찰서
1,2018,총계,소계,591625,28116,83062,65465,33239,31891,98977,76418,38227,72389,63841


In [180]:
df_bad_cars['년도'] = df_bad_cars['년도'].str.split('.').str[0]
df_bad_cars.head(2)

,년도,구분,경찰서별,합계,중부경찰서,동부경찰서,서부경찰서,남부경찰서,북부경찰서,수성경찰서,달서경찰서,성서경찰서,달성경찰서,강북경찰서
1,2018,총계,소계,591625,28116,83062,65465,33239,31891,98977,76418,38227,72389,63841
2,2018,위반사항,신호위반,93999,1618,13629,4703,3747,7589,20824,19523,7945,11716,2705


In [181]:
statioin_mapper = {
  '중부경찰서': '중구',
  '동부경찰서': '동구',
  '서부경찰서': '서구',
  '남부경찰서': '남구',
  '북부경찰서': '북구',
  '수성경찰서': '수성구',
  '달서경찰서': '달서구',
  '성서경찰서': '달서구',
  '달성경찰서': '달성군',
  '강북경찰서': '북구'
}

df_bad_cars = df_bad_cars.rename(columns=statioin_mapper)
df_bad_cars.head(1)

,년도,구분,경찰서별,합계,중구,동구,서구,남구,북구,수성구,달서구,달서구,달성군,북구
1,2018,총계,소계,591625,28116,83062,65465,33239,31891,98977,76418,38227,72389,63841


In [182]:
df_bad_cars = df_bad_cars.groupby(['구분','경찰서별', '년도']).sum().reset_index().T
df_bad_cars.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
구분,용도별,용도별,용도별,용도별,용도별,용도별,용도별,용도별,용도별,용도별,...,처리사항,처리사항,처리사항,처리사항,처리사항,처리사항,총계,총계,총계,총계
경찰서별,기타,기타,기타,기타,비사업용,비사업용,비사업용,비사업용,사업용,사업용,...,통고처분,통고처분,형사 입건,형사 입건,형사 입건,형사 입건,소계,소계,소계,소계
년도,2018,2019,2020,2021,2018,2019,2020,2021,2018,2019,...,2020,2021,2018,2019,2020,2021,2018,2019,2020,2021
합계,787,575,202,403,532398,540420,681876,672456,58440,64178,...,49193,62136,10082,8691,7466,7781,591625,605173,771947,764043
중구,56,55,-,-,23271,21856,-,-,4789,5086,...,-,-,686,598,-,-,28116,-,-,-


In [183]:
df_bad_cars = df_bad_cars.map(lambda x: x if x != '-' else np.nan).dropna(axis=1)
df_bad_cars

,0,1,4,5,8,9,12,16,20,24,...,55,56,59,60,61,72,73,76,77,80
구분,용도별,용도별,용도별,용도별,용도별,용도별,위반사항,위반사항,위반사항,위반사항,...,차종별,차종별,차종별,차종별,차종별,처리사항,처리사항,처리사항,처리사항,총계
경찰서별,기타,기타,비사업용,비사업용,사업용,사업용,과속,기타,무면허,신호위반,...,이륜차,이륜차,특수차,화물차,화물차,통고처분,통고처분,형사 입건,형사 입건,소계
년도,2018,2019,2018,2019,2018,2019,2018,2018,2018,2018,...,2018,2019,2018,2018,2019,2018,2019,2018,2019,2018
합계,787,575,532398,540420,58440,64178,446367,16525,2099,93999,...,5166,6972,2574,67231,67827,581543,596482,10082,8691,591625
중구,56,55,23271,21856,4789,5086,22452,1124,127,1618,...,117,919,105,3240,2160,27430,26399,686,598,28116
동구,98,70,75502,77527,7462,8942,63911,827,219,13629,...,712,789,374,8554,8284,81740,85526,1322,1013,83062
서구,76,57,57881,59767,7508,7736,55572,1009,356,4703,...,393,309,237,10091,10705,64337,66698,1128,862,65465
남구,55,39,29173,27347,4011,3966,25177,2352,128,3747,...,629,543,107,2419,2003,32624,30716,615,636,33239
북구,57,66,28826,29742,3008,4114,20557,1419,248,7589,...,256,492,153,3811,3946,31105,33095,786,827,31891
수성구,153,101,89834,76523,8990,8765,67317,4672,296,20824,...,1588,2290,380,8082,6774,97191,84010,1786,1379,98977


In [184]:
df_bad_cars = df_bad_cars.T.reset_index(drop=True)
df_bad_cars = df_bad_cars.drop('년도', axis=1).groupby(['구분', '경찰서별']).mean()
df_bad_cars.index = [f'{a}_{b}' for a,b in df_bad_cars.index]
df_bad_cars.head()

,합계,중구,동구,서구,남구,북구,수성구,달서구,달서구,달성군,북구
용도별_기타,681.0,55.5,84.0,66.5,47.0,61.5,127.0,82.0,76.5,39.0,42.0
용도별_비사업용,536409.0,22563.5,76514.5,58824.0,28260.0,29284.0,83178.5,70145.0,33670.5,73785.0,60184.0
용도별_사업용,61309.0,4937.5,8202.0,7622.0,3988.5,3561.0,8877.5,7796.0,4222.5,5901.5,6200.5
위반사항_과속,446367.0,22452.0,63911.0,55572.0,25177.0,20557.0,67317.0,47901.0,26797.0,58135.0,58548.0
위반사항_기타,16525.0,1124.0,827.0,1009.0,2352.0,1419.0,4672.0,3449.0,501.0,740.0,432.0


In [185]:
df_bad_cars.T.head()

,용도별_기타,용도별_비사업용,용도별_사업용,위반사항_과속,위반사항_기타,위반사항_무면허,위반사항_신호위반,위반사항_안전띠 미착용,위반사항_안전운전 의무불이행,위반사항_음주운전,위반사항_중앙선 침범,차종별_기타(특수차),차종별_승용차,차종별_승합차,차종별_이륜차,차종별_특수차,차종별_화물차,처리사항_통고처분,처리사항_형사 입건,총계_소계
합계,681.0,536409.0,61309.0,446367.0,16525.0,2099.0,93999.0,14223.0,5078.0,7682.0,5652.0,3357.0,506114.5,15721.0,6069.0,2574.0,67529.0,589012.5,9386.5,591625.0
중구,55.5,22563.5,4937.5,22452.0,1124.0,127.0,1618.0,1546.0,311.0,540.0,398.0,157.0,23487.5,720.0,518.0,105.0,2700.0,26914.5,642.0,28116.0
동구,84.0,76514.5,8202.0,63911.0,827.0,219.0,13629.0,1656.0,721.0,1070.0,1029.0,570.0,72887.0,2272.0,750.5,374.0,8419.0,83633.0,1167.5,83062.0
서구,66.5,58824.0,7622.0,55572.0,1009.0,356.0,4703.0,2387.0,427.0,744.0,267.0,340.0,53583.0,1892.0,351.0,237.0,10398.0,65517.5,995.0,65465.0
남구,47.0,28260.0,3988.5,25177.0,2352.0,128.0,3747.0,768.0,310.0,467.0,290.0,106.0,28691.5,700.5,586.0,107.0,2211.0,31670.0,625.5,33239.0


In [186]:
df_bad_cars.T.reset_index().groupby('index').sum().drop('합계').reset_index().to_csv(f'../../main/data/origin/additionals/bad_cars.csv')

In [187]:
gungu = gpd.read_file('./externals/대구_빅데이터_활용센터/8_1_jiri_sigungu.shp', encoding='cp949')
gungu

,base_year,sigungu_cd,sigungu_nm,geometry
0,2018,22010,중구,"POLYGON ((1097857.218 1765177.561, 1097872.134..."
1,2018,22020,동구,"POLYGON ((1107777.596 1780522.263, 1107845.409..."
2,2018,22030,서구,"POLYGON ((1094929.282 1767244.458, 1094939.136..."
3,2018,22040,남구,"POLYGON ((1097010.572 1763145.859, 1097030.717..."
4,2018,22050,북구,"POLYGON ((1100413.419 1776575.012, 1100415.868..."
5,2018,22060,수성구,"POLYGON ((1103905.976 1765250.085, 1103921.812..."
6,2018,22070,달서구,"POLYGON ((1091387.729 1763977.918, 1091424.370..."
7,2018,22310,달성군,"MULTIPOLYGON (((1089667.472 1758560.378, 10896..."


In [188]:
gungu['area'] = gungu['geometry'].map(lambda x: x.area)
gungu

,base_year,sigungu_cd,sigungu_nm,geometry,area
0,2018,22010,중구,"POLYGON ((1097857.218 1765177.561, 1097872.134...",7.042380e+06
1,2018,22020,동구,"POLYGON ((1107777.596 1780522.263, 1107845.409...",1.814826e+08
2,2018,22030,서구,"POLYGON ((1094929.282 1767244.458, 1094939.136...",1.740374e+07
3,2018,22040,남구,"POLYGON ((1097010.572 1763145.859, 1097030.717...",1.779893e+07
4,2018,22050,북구,"POLYGON ((1100413.419 1776575.012, 1100415.868...",9.374851e+07
5,2018,22060,수성구,"POLYGON ((1103905.976 1765250.085, 1103921.812...",7.678767e+07
6,2018,22070,달서구,"POLYGON ((1091387.729 1763977.918, 1091424.370...",6.251677e+07
7,2018,22310,달성군,"MULTIPOLYGON (((1089667.472 1758560.378, 10896...",4.228042e+08


In [189]:
df_bad_cars.T

,용도별_기타,용도별_비사업용,용도별_사업용,위반사항_과속,위반사항_기타,위반사항_무면허,위반사항_신호위반,위반사항_안전띠 미착용,위반사항_안전운전 의무불이행,위반사항_음주운전,위반사항_중앙선 침범,차종별_기타(특수차),차종별_승용차,차종별_승합차,차종별_이륜차,차종별_특수차,차종별_화물차,처리사항_통고처분,처리사항_형사 입건,총계_소계
합계,681.0,536409.0,61309.0,446367.0,16525.0,2099.0,93999.0,14223.0,5078.0,7682.0,5652.0,3357.0,506114.5,15721.0,6069.0,2574.0,67529.0,589012.5,9386.5,591625.0
중구,55.5,22563.5,4937.5,22452.0,1124.0,127.0,1618.0,1546.0,311.0,540.0,398.0,157.0,23487.5,720.0,518.0,105.0,2700.0,26914.5,642.0,28116.0
동구,84.0,76514.5,8202.0,63911.0,827.0,219.0,13629.0,1656.0,721.0,1070.0,1029.0,570.0,72887.0,2272.0,750.5,374.0,8419.0,83633.0,1167.5,83062.0
서구,66.5,58824.0,7622.0,55572.0,1009.0,356.0,4703.0,2387.0,427.0,744.0,267.0,340.0,53583.0,1892.0,351.0,237.0,10398.0,65517.5,995.0,65465.0
남구,47.0,28260.0,3988.5,25177.0,2352.0,128.0,3747.0,768.0,310.0,467.0,290.0,106.0,28691.5,700.5,586.0,107.0,2211.0,31670.0,625.5,33239.0
북구,61.5,29284.0,3561.0,20557.0,1419.0,248.0,7589.0,787.0,363.0,517.0,411.0,205.0,27648.0,827.0,374.0,153.0,3878.5,32100.0,806.5,31891.0
수성구,127.0,83178.5,8877.5,67317.0,4672.0,296.0,20824.0,2469.0,857.0,1390.0,1152.0,465.0,80332.0,2061.5,1939.0,380.0,7428.0,90600.5,1582.5,98977.0
달서구,82.0,70145.0,7796.0,47901.0,3449.0,190.0,19523.0,3243.0,544.0,938.0,630.0,309.0,67211.0,1872.0,731.5,333.0,7887.5,76962.5,1060.5,76418.0
달서구,76.5,33670.5,4222.5,26797.0,501.0,267.0,7945.0,574.0,673.0,852.0,618.0,184.0,32363.0,1038.0,312.5,162.0,4083.0,36938.5,1031.0,38227.0
달성군,39.0,73785.0,5901.5,58135.0,740.0,152.0,11716.0,385.0,369.0,498.0,394.0,666.0,64176.0,2526.5,239.5,469.0,12216.0,78998.5,727.0,72389.0


In [190]:
df_bad_cars.T

,용도별_기타,용도별_비사업용,용도별_사업용,위반사항_과속,위반사항_기타,위반사항_무면허,위반사항_신호위반,위반사항_안전띠 미착용,위반사항_안전운전 의무불이행,위반사항_음주운전,위반사항_중앙선 침범,차종별_기타(특수차),차종별_승용차,차종별_승합차,차종별_이륜차,차종별_특수차,차종별_화물차,처리사항_통고처분,처리사항_형사 입건,총계_소계
합계,681.0,536409.0,61309.0,446367.0,16525.0,2099.0,93999.0,14223.0,5078.0,7682.0,5652.0,3357.0,506114.5,15721.0,6069.0,2574.0,67529.0,589012.5,9386.5,591625.0
중구,55.5,22563.5,4937.5,22452.0,1124.0,127.0,1618.0,1546.0,311.0,540.0,398.0,157.0,23487.5,720.0,518.0,105.0,2700.0,26914.5,642.0,28116.0
동구,84.0,76514.5,8202.0,63911.0,827.0,219.0,13629.0,1656.0,721.0,1070.0,1029.0,570.0,72887.0,2272.0,750.5,374.0,8419.0,83633.0,1167.5,83062.0
서구,66.5,58824.0,7622.0,55572.0,1009.0,356.0,4703.0,2387.0,427.0,744.0,267.0,340.0,53583.0,1892.0,351.0,237.0,10398.0,65517.5,995.0,65465.0
남구,47.0,28260.0,3988.5,25177.0,2352.0,128.0,3747.0,768.0,310.0,467.0,290.0,106.0,28691.5,700.5,586.0,107.0,2211.0,31670.0,625.5,33239.0
북구,61.5,29284.0,3561.0,20557.0,1419.0,248.0,7589.0,787.0,363.0,517.0,411.0,205.0,27648.0,827.0,374.0,153.0,3878.5,32100.0,806.5,31891.0
수성구,127.0,83178.5,8877.5,67317.0,4672.0,296.0,20824.0,2469.0,857.0,1390.0,1152.0,465.0,80332.0,2061.5,1939.0,380.0,7428.0,90600.5,1582.5,98977.0
달서구,82.0,70145.0,7796.0,47901.0,3449.0,190.0,19523.0,3243.0,544.0,938.0,630.0,309.0,67211.0,1872.0,731.5,333.0,7887.5,76962.5,1060.5,76418.0
달서구,76.5,33670.5,4222.5,26797.0,501.0,267.0,7945.0,574.0,673.0,852.0,618.0,184.0,32363.0,1038.0,312.5,162.0,4083.0,36938.5,1031.0,38227.0
달성군,39.0,73785.0,5901.5,58135.0,740.0,152.0,11716.0,385.0,369.0,498.0,394.0,666.0,64176.0,2526.5,239.5,469.0,12216.0,78998.5,727.0,72389.0


In [191]:
df_bad_cars = df_bad_cars.reset_index().groupby('index').sum().reset_index().drop(columns=['합계']).set_index('index')
df_bad_cars

,중구,동구,서구,남구,북구,수성구,달서구,달서구,달성군,북구
index,,,,,,,,,,
용도별_기타,55.5,84.0,66.5,47.0,61.5,127.0,82.0,76.5,39.0,42.0
용도별_비사업용,22563.5,76514.5,58824.0,28260.0,29284.0,83178.5,70145.0,33670.5,73785.0,60184.0
용도별_사업용,4937.5,8202.0,7622.0,3988.5,3561.0,8877.5,7796.0,4222.5,5901.5,6200.5
위반사항_과속,22452.0,63911.0,55572.0,25177.0,20557.0,67317.0,47901.0,26797.0,58135.0,58548.0
위반사항_기타,1124.0,827.0,1009.0,2352.0,1419.0,4672.0,3449.0,501.0,740.0,432.0
위반사항_무면허,127.0,219.0,356.0,128.0,248.0,296.0,190.0,267.0,152.0,116.0
위반사항_신호위반,1618.0,13629.0,4703.0,3747.0,7589.0,20824.0,19523.0,7945.0,11716.0,2705.0
위반사항_안전띠 미착용,1546.0,1656.0,2387.0,768.0,787.0,2469.0,3243.0,574.0,385.0,408.0
위반사항_안전운전 의무불이행,311.0,721.0,427.0,310.0,363.0,857.0,544.0,673.0,369.0,503.0


In [192]:
df_bad_cars = df_bad_cars.T.reset_index().groupby('index').sum().reset_index()

In [193]:
def calc_mildo_by_area(row: pd.Series):
  row = {**row}
  area = gungu.loc[gungu['sigungu_nm'] == row['index']]
  if len(area) > 0:
    for col in df_bad_cars.columns:
      if col == 'index': continue
      row[col] = row[col]/area['area'].iloc[0]
    return pd.Series(row)
  print(row['index'])
  return row

In [196]:
df_bad_cars = df_bad_cars.apply(
  calc_mildo_by_area,
  axis=1
).reset_index(drop=True)
df_bad_cars

,index,용도별_기타,용도별_비사업용,용도별_사업용,위반사항_과속,위반사항_기타,위반사항_무면허,위반사항_신호위반,위반사항_안전띠 미착용,위반사항_안전운전 의무불이행,...,위반사항_중앙선 침범,차종별_기타(특수차),차종별_승용차,차종별_승합차,차종별_이륜차,차종별_특수차,차종별_화물차,처리사항_통고처분,처리사항_형사 입건,총계_소계
0,남구,2.640609e-06,0.001588,0.000224,0.001415,0.000132,7.191445e-06,0.000211,4.314867e-05,1.741678e-05,...,1.629312e-05,0.000006,0.001612,0.000039,3.292334e-05,0.000006,0.000124,0.001779,0.000035,0.001867
1,달서구,2.535320e-06,0.001661,0.000192,0.001195,0.000063,7.310038e-06,0.000439,6.105561e-05,1.946678e-05,...,1.996264e-05,0.000008,0.001593,0.000047,1.669952e-05,0.000008,0.000191,0.001822,0.000033,0.001834
2,달성군,9.224128e-08,0.000175,0.000014,0.000137,0.000002,3.595045e-07,0.000028,9.105870e-07,8.727444e-07,...,9.318734e-07,0.000002,0.000152,0.000006,5.664561e-07,0.000001,0.000029,0.000187,0.000002,0.000171
3,동구,4.628543e-07,0.000422,0.000045,0.000352,0.000005,1.206727e-06,0.000075,9.124843e-06,3.972833e-06,...,5.669966e-06,0.000003,0.000402,0.000013,4.135383e-06,0.000002,0.000046,0.000461,0.000006,0.000458
4,북구,1.104018e-06,0.000954,0.000104,0.000844,0.000020,3.882728e-06,0.000110,1.274687e-05,9.237480e-06,...,9.322815e-06,0.000006,0.000889,0.000028,6.837442e-06,0.000004,0.000130,0.001043,0.000017,0.001021
5,서구,3.821019e-06,0.003380,0.000438,0.003193,0.000058,2.045538e-05,0.000270,1.371545e-04,2.453496e-05,...,1.534153e-05,0.000020,0.003079,0.000109,2.016808e-05,0.000014,0.000597,0.003765,0.000057,0.003762
6,수성구,1.653911e-06,0.001083,0.000116,0.000877,0.000061,3.854786e-06,0.000271,3.215360e-05,1.116065e-05,...,1.500241e-05,0.000006,0.001046,0.000027,2.525145e-05,0.000005,0.000097,0.001180,0.000021,0.001289
7,중구,7.880859e-06,0.003204,0.000701,0.003188,0.000160,1.803368e-05,0.000230,2.195281e-04,4.416121e-05,...,5.651499e-05,0.000022,0.003335,0.000102,7.355468e-05,0.000015,0.000383,0.003822,0.000091,0.003992


In [197]:
df_bad_cars.to_csv('../../main/data/origin/additionals/bad_cars.csv')